-------

#  Reading the CSV file 

Reading the csv file  and printing the information about the data

In [114]:
#libraries used
import pandas as pd
import numpy as np
from scipy.stats import skew

We will get warning because we have mixed data types due to "?" in the data. 

In [115]:
# reading csv
df_initial = pd.read_csv("project-2018-BRFSS-arthritis.csv")

/tmp/ipykernel_4016/4112109159.py:2: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_initial = pd.read_csv("project-2018-BRFSS-arthritis.csv")


Normally we should not see any categorical data because all of our values are already converted to numeric. As we see above error we have mixed data types because of the "?" symbol

In [116]:
print("Count of Tuples: ", len(df_initial))
print("Count of Attributes variables: ", len(df_initial.columns))
print("there are ", len(df_initial.select_dtypes(exclude='number').columns), " categorical attributes")
#df_initial.select_dtypes(exclude='number').columns

Count of Tuples:  11933
Count of Attributes variables:  108
there are  37  categorical attributes


-------

# Replace "?" with null values

We want to use pandas special functions to fill the null values. Therefore we will replace them with nan

In [117]:
# saving as a different df to protect our original df 
df_modified_null = df_initial
df_modified_null= df_modified_null.replace("?",np.nan)


In [118]:
print("any value is null", df_modified_null.isnull().values.any())
print("total tuples that missing ",df_modified_null.isnull().sum().sum())
missing_tuples = df_modified_null.isnull().sum()
missing_tuples= pd.DataFrame({'name':missing_tuples.index, 'total_missing_columns':missing_tuples.values})
notnull = df_modified_null.notna().sum()
notnull  =pd.DataFrame({'name':notnull.index, 'non_missing':notnull.values})

any value is null True
total tuples that missing  14830


In [119]:
data_info= missing_tuples.merge(notnull, how='left')
size= len(df_initial)
data_info["percentage_missing"]= round(data_info['total_missing_columns']/size, 2)
print("total columns that has missing values: ", len(data_info[data_info["total_missing_columns"]!=0]))
#data_info[data_info["total_missing_columns"]!=0]


total columns that has missing values:  37


Creating meta data to help us to evaluate the model better 

In [120]:
my_list= []
for items in data_info.name:
    my_list.append([df_modified_null[items].value_counts().to_dict()])

data_info["unique_items"]= my_list
data_info

,name,total_missing_columns,non_missing,percentage_missing,unique_items
0,x.aidtst3,805,11128,0.07,"[{'2': 7002, '1': 3657, '9': 469}]"
1,employ1,34,11899,0.00,"[{'1': 4964, '7': 3505, '2': 1091, '8': 869, '..."
2,income2,114,11819,0.01,"[{'8': 3343, '7': 1592, '6': 1382, '99': 1117,..."
3,weight2,180,11753,0.02,"[{'180': 564, '200': 535, '150': 514, '160': 4..."
4,height3,198,11735,0.02,"[{'506': 1050, '504': 966, '507': 950, '505': ..."
...,...,...,...,...,...
103,x.michd,114,11819,0.01,"[{'2': 10748, '1': 1071}]"
104,x.ltasth1,0,11933,0.00,"[{1: 10233, 2: 1662, 9: 38}]"
105,x.casthm1,0,11933,0.00,"[{1: 10708, 2: 1132, 9: 93}]"
106,x.state,0,11933,0.00,"[{36: 936, 24: 487, 27: 464, 12: 417, 31: 382,..."


**Converting all values to numeric **

In [121]:
#data_info[data_info["total_missing_columns"]!=0].name.to_list()
df_2= df_modified_null
missing_columns= data_info[data_info["total_missing_columns"]!=0].name.to_list()

for item in missing_columns:
    df_2[item] =pd.to_numeric(df_2[item])



Double checking to see if there is any mixed values and if all values are numeric

In [122]:
print("there are ", len(df_2.select_dtypes(exclude='number').columns), " categorical attributes")
df_2.select_dtypes(exclude='number').columns

there are  0  categorical attributes


Index([], dtype='object')

We are looking to see what percentage of the data is missing over all and tuple based. Are there any tuples that missing 60% data? 

In [123]:
total_percentage_missing = round(df_modified_null.isnull().sum().sum()/len( df_initial),2)
print("Over all total percentage that is missing", total_percentage_missing, "percent")
print(data_info.percentage_missing[data_info.percentage_missing>0.05])
print("total ", len(data_info.percentage_missing[data_info.percentage_missing>0.05]), "tuples")

Over all total percentage that is missing 1.24 percent
0     0.07
17    0.07
18    0.07
57    0.07
58    0.08
59    0.08
83    0.23
Name: percentage_missing, dtype: float64
total  7 tuples


Since there is only seven tuples left I will keep them but I might delete them in later clean up

In [129]:
#df_2_mod = df_2[~df_2.index.isin(data_info.percentage_missing[data_info.percentage_missing>0.05].index)]


------

Missing data treatment

We checked our missing data and find out actual numeric data and we will use median info to update them . We use median because there are outliers and mean is effected by outliers

In [136]:

# updating categorical variables with median
# we are using the median because the data includes a lot of outliers
df_3= df_2
df_3[['weight2', 'height3', 'htin4', 'wtkg3',  'x.bmi5',  'htm4']] = df_3[['weight2', 'height3', 'htin4', 'wtkg3',  'x.bmi5',  'htm4']].fillna(df_3[['weight2', 'height3', 'htin4', 'wtkg3',  'x.bmi5',  'htm4']].median())


for categorical variables we will use mode to find out most repeated values

In [137]:
#df_3.loc[ : , ~df_3.columns.isin(['weight2', 'height3', 'htin4', 'wtkg3',  'x.bmi5', 'htm4'])]
y = df_3.loc[ : , ~df_3.columns.isin(['weight2', 'height3', 'htin4', 'wtkg3',  'x.bmi5', 'htm4'])]
my_list = y.columns.to_list()
for item in my_list:
    df_3[item] = df_3[item].fillna(df_3[item].mode()[0])



We will double check to see if we have any missing values 

In [138]:
print("any value is null", df_3.isnull().values.any())
print("total tuples that missing ",df_3.isnull().sum().sum())


any value is null False
total tuples that missing  0


----

outliers treatment

We do not want to delete data because of outliers because we might have some useful data. Therefore we will only update out extreme data points with median of the data for numeric values. 

In [139]:
# size is bigger than ten
count= 0
cat_type= []
for item in range(len(data_info)):
    size =len(data_info.unique_items[item][0])
    if size > 11:
        count +=1
        cat_type.append("numeric")
        #print('a')
    else:
        cat_type.append("categorical")

data_info["category_type"]= cat_type
print(count)


28


In [140]:
# The 3(IQR) Criterion for Outliers
def outliersTreatment(df, col_name,whisker_width ):
    median = df[col_name].median()
    Q1=df[col_name].quantile(q=0.25)
    Q3=df[col_name].quantile(q=0.75)
    IQR= Q3-Q1
    lower_threshold = Q1 - (whisker_width*IQR)
    upper_threshold = Q3 + (whisker_width*IQR)
    print(lower_threshold)
    print(upper_threshold)
    df[col_name] = np.where(df[col_name] > upper_threshold, median, df[col_name])
    df[col_name] = np.where(df[col_name] < lower_threshold, median, df[col_name])
    return df
   


we change some of the numeric columns outliers but there might be more. 

we will get back again clean more 

In [ ]:
df_4 = outliersTreatment(df=df_3, col_name='weight2', whisker_width=3)

df_5= outliersTreatment(df=df_4, col_name='height3', whisker_width=3)

df_6= outliersTreatment(df=df_5, col_name='htin4', whisker_width=3)

df_7= outliersTreatment(df=df_6, col_name='x.bmi5', whisker_width=3)

# bmi seems like a calculated field but there is no time to chec 



0.0
350.0
492.0
520.0
46.0
88.0
496.0
5004.0


In [141]:
df_8= outliersTreatment(df=df_7, col_name='htm4', whisker_width=3)
print(type(df_7.htm4))

118.0
223.0
<class 'pandas.core.series.Series'>


we save our data frame to csv file ready for processing. 

In [142]:
df_8.to_csv("HicranArnold_project_intial.csv", index=False)

In [143]:
len(df_8)

11933

In [ ]:
# data_dropped = data_info.drop(['unique_items'], axis=1)
# data_dropped.to_csv("metadata.csv")